In [1]:
!pip install -q gpt_utils
!pip install -q pandas
!pip install -q cuda-toolkit-11.3

ERROR: Could not find a version that satisfies the requirement cuda-toolkit-11.3 (from versions: none)
ERROR: No matching distribution found for cuda-toolkit-11.3


In [2]:
import pandas as pd
import numpy as np
from tokenizers.models import BPE
from tokenizers.trainers import *
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
import torch
from datasets import Dataset
from gpt_utils import *
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
from ipywidgets import *
from tqdm import tqdm
tqdm.pandas()
import RNA
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from transformers import Trainer, GPT2LMHeadModel, AutoTokenizer
import transformers
from tqdm import tqdm

In [3]:
def is_valid_dot_bracket(structure, sequence):
    stack = []
    valid_pairs = [('T', 'G'), ('G', 'T'), ('C', 'G'), ('G', 'C'), ('A', 'T'), ('T', 'A')]
    invalid_pairs_dict = {}
    
    for i, char in enumerate(structure):
        if char == '(':
            stack.append((i, sequence[i]))
        elif char == ')':
            if not stack:
                return False, invalid_pairs_dict
            else:
                position, nucleotide = stack.pop()
                if (nucleotide, sequence[i]) not in valid_pairs:
                    invalid_pair = f"{nucleotide}{sequence[i]}"
                    invalid_pairs_dict[invalid_pair] = invalid_pairs_dict.get(invalid_pair, 0) + 1
                    
    # Check if there are any unmatched parentheses left
    if stack:
        return False, invalid_pairs_dict
    
    # Check if any invalid pairs were found
    if invalid_pairs_dict:
        return False, invalid_pairs_dict
        
    return True, invalid_pairs_dict

def find_closing_bracket(struct_list):
    # This function finds the index of the matching closing bracket.
    depth = 1
    for i, char in enumerate(struct_list):
        if char == '(':
            depth += 1
        elif char == ')':
            depth -= 1
            if depth == 0:
                return i
    raise ValueError("No matching closing bracket found")
    
def mature_or_star(row):
    full_seq = row['full_seq']
    mature_start = full_seq.find(row['Mature'])
    star_start = full_seq.find(row['Star'])
    star_end = star_start + len(row['Star'])
    
    encoded_seq = full_seq
    if star_start < star_end:
        if mature_start < star_start:
            mature_sequences.append(encoded_seq)
        else:
            star_sequences.append(encoded_seq)

def encode_rna(row):
    full_seq = row[relevant_seq]
    mature_start = full_seq.find(row['Mature'])
    if mature_start == -1:
        print("no mature")
    mature_end = mature_start + len(row['Mature'])
    star_start = full_seq.find(row['Star'])
    if star_start == -1:
        print("no star")
    star_end = star_start + len(row['Star'])
    # star_end = len(row['Star'])

    encoded_seq = full_seq
    if star_start < star_end:
        if mature_start < star_start:
            encoded_seq = (encoded_seq[:mature_start] + 'ZZZZZ' +
                           encoded_seq[mature_start:mature_end] + 'BBBBB' +
                           encoded_seq[mature_end:star_start] + 'DDDDD' +
                           encoded_seq[star_start:star_end] + 'FFFFF' +
                           encoded_seq[star_end:])
        else:
            encoded_seq = (encoded_seq[:star_start] + 'DDDDD' +
                           encoded_seq[star_start:star_end] + 'FFFFF' +
                           encoded_seq[star_end:mature_start] + 'ZZZZZ' +
                           encoded_seq[mature_start:mature_end] + 'BBBBB' +
                           encoded_seq[mature_end:])
    return encoded_seq

def decode_rna(encoded_seq):
    decoded_seq=  ['']*1000
    # decoded_struct = ['']*1000
    pair_dict_reverse = {'V': 'GC', 'W': 'CG','X': 'AT', 'Y': 'TA','N': 'GT', 'M': 'TG'}
    
    curr_index = len(encoded_seq) - 1
    i=0
    while curr_index >= 0:
        char = encoded_seq[curr_index]
        if char.isdigit():
            # distance can be 2 or 3 consecutive digits from curr_index
            if encoded_seq[curr_index-2].isdigit():
                distance = encoded_seq[curr_index-2:curr_index+1]
                curr_index -= 3
            else:
                distance = encoded_seq[curr_index-1:curr_index+1]
                curr_index -= 2
            pair =  pair_dict_reverse.get(encoded_seq[curr_index])
            base1,base2 = pair[0],pair[1]
            decoded_seq.insert(i,base1) #; decoded_struct.insert(i,'(')
            # print(decoded_seq)
            decoded_seq.insert(i-int(distance),base2) #; decoded_struct.insert(i-int(distance),')')
            i+=2

        else:
            decoded_seq.insert(i,encoded_seq[curr_index])
            i+=1
        curr_index-=1

    decoded_seq = ''.join(decoded_seq)[::-1]

    return decoded_seq#, decoded_struct

def extra_decode_rna(encoded_seq):
    # Decoding logic you provided
    decoded_seq = ['']*1000
    decoded_struct = ['']*1000
    pair_dict_reverse = {'V': 'GC', 'W': 'CG','X': 'AT', 'Y': 'TA','N': 'GT', 'M': 'TG'}

    curr_index = len(encoded_seq) - 1
    i=0
    while curr_index >= 0:
        char = encoded_seq[curr_index]
        if char.isdigit():
            # distance can be 2 or 3 consecutive digits from curr_index
            if encoded_seq[curr_index-2].isdigit():
                distance = encoded_seq[curr_index-2:curr_index+1]
                curr_index -= 3
            else:
                distance = encoded_seq[curr_index-1:curr_index+1]
                curr_index -= 2
            pair =  pair_dict_reverse.get(encoded_seq[curr_index])
            base1,base2 = pair[0],pair[1]
            decoded_seq.insert(i,base1) ; decoded_struct.insert(i,'(')
            decoded_seq.insert(i-int(distance),base2) ; decoded_struct.insert(i-int(distance),')')
            i+=2

        else:
            decoded_seq.insert(i,encoded_seq[curr_index])
            # print(i,encoded_seq[curr_index])
            decoded_struct.insert(i,'.')
            i+=1
        curr_index-=1

    decoded_seq = ''.join(decoded_seq)[::-1]
    decoded_struct = ''.join(decoded_struct)[::-1]

    # Replace T with U in the RNA sequence
    decoded_seq = decoded_seq.replace('T', 'U')
    
    # Extract the mature and star sections
    mature_start = decoded_seq.find('ZZZZZ') + 5
    mature_end = decoded_seq.find('BBBBB')
    star_start = decoded_seq.find('DDDDD') + 5
    star_end = decoded_seq.find('FFFFF')
    
    # Extract the mature and star sections
    mature_section = decoded_seq[mature_start:mature_end]
    star_section = decoded_seq[star_start:star_end]

    # Remove the markers and construct the full sequence structure
    if mature_start > star_start:
        # If mature section appears before the star section
        full_seq_struct = (decoded_struct[:star_start - 5] +
                           decoded_struct[star_start :star_end] + 
                           decoded_struct[star_end+5 :mature_start - 5] + 
                           decoded_struct[mature_start:mature_end]+ 
                           decoded_struct[mature_end+5:])
    else:
        # If mature section appears before the star section
        full_seq_struct = (decoded_struct[:mature_start - 5] +
                           decoded_struct[mature_start :mature_end] + 
                           decoded_struct[mature_end+5 :star_start - 5] + 
                           decoded_struct[star_start:star_end]+ 
                           decoded_struct[star_end+5:])

    # Remove the markers and replace 'T' with 'U' in the full sequence
    full_seq = decoded_seq.replace('ZZZZZ', '').replace('BBBBB', '').replace('DDDDD', '').replace('FFFFF', '')

    return full_seq, full_seq_struct, mature_section, star_section


In [4]:
def fasta_to_tuples(filename):
    with open(filename, 'r') as f:
        content = f.read().strip().split("\n")

    sequences = []
    for i in range(0, len(content), 2):
        header = content[i][1:]  # remove '>'
        sequence = content[i+1]
        sequences.append((header, sequence))

    return sequences

def tuples_to_jsonl(sequences, output_filename):
    with open(output_filename, 'w') as f:
        for header, sequence in sequences:
            data = {
                "id": header,
                "sequence": sequence
            }
            f.write(json.dumps(data) + '\n')

def calculate_statistics(sequences):
    num_sequences = len(sequences)
    avg_length = int(sum(len(seq) for seq in sequences) / num_sequences)
    min_length = min(len(seq) for seq in sequences)
    max_length = max(len(seq) for seq in sequences)

    return {
        "Number of sequences": num_sequences,
        "Average sequence length": avg_length,
        "Min sequence length": min_length,
        "Max sequence length": max_length
    }


### Load Human Data

#### All data

In [5]:
all_data = fasta_to_tuples("/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/Data_source/miRGeneDB/precursors_all_species_w_flank.fa")
original_human_sequences = [s[1] for s in all_data  if s[0].startswith('Hsa')]

preprocess_data = pd.read_csv("/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/Data_source/Data_output/miRGeneDB_output/miRGeneDB_features.csv")

human_data = preprocess_data[preprocess_data['full_seq'].isin(original_human_sequences)]
print("len after is in: ", len(human_data), len(original_human_sequences)) # duplicates (?)

human_data = human_data.drop_duplicates(subset=['full_seq'], keep='first')
print("len after remove duplicates", len(human_data)) # after drop duplicates


# ** without flanks: ** - if you want with flank, make the next lines in comment
human_data_fasta = fasta_to_tuples("/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/Data_source/miRGeneDB/precursors_human_no_flank.fas.txt")
original_human_sequences = [s[1] for s in human_data_fasta]

## remove flanks from sequences
human_data = human_data.dropna(subset=['full_seq', 'flank1', 'flank2']); # found some null flanks
for index, row in human_data.iterrows():
    human_data.at[index, 'full_seq'] = human_data.at[index, 'full_seq'].replace(row['flank1'], '').replace(row['flank2'], '')
print("len after removing flanks: ", len(human_data)) # after drop duplicates


len after is in:  701 567
len after remove duplicates 539
len after removing flanks:  530


In [6]:
relevant_seq = 'full_seq'

### Train set

In [7]:
# True if no flanks
hairpin_bool = False
if hairpin_bool:
    relevant_seq = 'pre_mirna'
else:
    relevant_seq = 'full_seq'

In [8]:
human_data = pd.read_csv("/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/seq_clusters/human_train_data.csv")
original_human_sequences = human_data[relevant_seq].tolist()

In [9]:
human_data = human_data.dropna(subset=['Star','Mature']) # found some null star and mature

human_data.loc[:, 'encoded_seq'] = human_data.apply(encode_rna, axis=1)
len_before = len(human_data)
human_data = human_data[human_data['encoded_seq'] != -1]
print(f"Removed {len_before - len(human_data)} rows with invalid dot-bracket language")

human_data['decoded_seq'] =  human_data['encoded_seq'].apply(decode_rna) #zip(*human_data['encoded_seq'].progress_apply(decode_rna)) #,human_data['decoded_struct']
human_data['decoded_seq'] =  human_data['decoded_seq'].apply(lambda x: x.replace('DDDDD','').replace('FFFFF','').replace('ZZZZZ','').replace('BBBBB',''))

print(np.equal(human_data['decoded_seq'], human_data[relevant_seq]).value_counts())


index_to_remove = []
for i in range(len(human_data)):
    if list(human_data['decoded_seq'])[i] != list(human_data[relevant_seq])[i]:
        index_to_remove.append(i)
human_data.drop(human_data.index[index_to_remove], inplace=True)
print(len(human_data),f'total of {len(index_to_remove)} removed')


Removed 0 rows with invalid dot-bracket language
True    389
Name: count, dtype: int64
389 total of 0 removed


In [10]:
encoded_human_sequences = ['<SOS>' + sequence + '<EOS>' for sequence in human_data['encoded_seq']]

In [11]:
calculate_statistics(encoded_human_sequences)

{'Number of sequences': 389,
 'Average sequence length': 149,
 'Min sequence length': 142,
 'Max sequence length': 156}

### Train New BPE tokenizer

**no need to train, use the tokenizer from the beggining of the training**

In [12]:
def token_statistics(tokenizer, sequences):
    token_lengths = []

    for seq in sequences:
        tokens = tokenizer.encode(seq).tokens
        token_lengths.extend([len(token) for token in tokens])

    avg_length = np.mean(token_lengths)
    min_length = np.min(token_lengths)
    max_length = np.max(token_lengths)
    median_length = np.median(token_lengths)
    std_dev = np.std(token_lengths)

    return {
        "average": avg_length,
        "min": min_length,
        "max": max_length,
        "median": median_length,
        "std_dev": std_dev
    }

### Load BPE tokenizer

In [13]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file="GPT_mature_star_bpe_hairpin_tokenizer.json") # GPT_mature_star_bpe_hairpin_tokenizer / "gpt_rna_fine_tuned_ms/tokenizer.json" "GPT_mature_star_bpe_hairpin_tokenizer.json"# loading the tokenizer from the pretrained training with the encoding of mature+star
# tokenizer = AutoTokenizer.from_pretrained("gpt_rna_mirgene_fine_tuned_no_flanks_ms")  # no flanks!

# Define special tokens
special_tokens = {
    "pad_token": "<PAD>",
    "bos_token": "<SOS>",
    "eos_token": "<EOS>",
    "unk_token": "<UNK>"
}

# Add special tokens to the tokenizer
tokenizer.add_special_tokens(special_tokens)

MAX_SEQ_LEN = max([len(s) for s in encoded_human_sequences]) 

def tokenize_function(data):
    # Tokenize the sequences
    output = tokenizer(data["sequences"], truncation=True, padding='max_length', max_length=MAX_SEQ_LEN)
    # Use input_ids as labels
    output["labels"] = output["input_ids"].copy()
    return output

dataset = Dataset.from_dict({"sequences": encoded_human_sequences})

tokenized_human_data = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/389 [00:00<?, ? examples/s]

In [15]:
print(len(tokenized_human_data))

389


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Fine tune on Human

In [20]:
# fine tune on human, based on gff and mirgeneDB
# Load the pre-trained model from Hugging Face
model_name = "mirgene_ms_all_data_hairpin"  # mirgene_ms_hairpin
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define fine-tuning arguments with a smaller learning rate
fine_tuning_args = TrainingArguments(
    output_dir='./human_ms_all_data_hairpin_results',   # / human_ms_hairpin_all_data_results / human_ms_hairpin_results
    num_train_epochs=80,
    per_device_train_batch_size=8,
    logging_dir='./human_ms_all_data_hairpin_logs',   # / human_ms_hairpin_logs
    learning_rate=1e-5,  # smaller learning rate for fine-tuning
)

# Create a trainer instance for fine-tuning
fine_tuning_trainer = Trainer(
    model=model,
    args=fine_tuning_args,
    train_dataset= tokenized_human_data,
)

# Fine-tune the model
fine_tuning_trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("human_ms_all_data_hairpin_mirgendb_gff")  #human_ms_hairpin_mirgendb_gff

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
500,0.124100
1000,0.073700
1500,0.066200
2000,0.063400
2500,0.062000
3000,0.060900
3500,0.060200
4000,0.059500
4500,0.059100
5000,0.058600


## Generate sequences

In [21]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file="GPT_mature_star_bpe_hairpin_tokenizer.json") #GPT_mature_star_bpe_hairpin_tokenizer

### Generate with prefix

In [18]:
# Set logging level to error to avoid the informational logs
transformers.logging.set_verbosity_error()

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("human_ms_mirgendb_gff") # human_ms_hairpin_mirgendb_gff / gpt_fine_tuned_human_mirgendb_gff

# Ensure that the tokenizer's pad_token is set if it's not already
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

avg_length = calculate_statistics(original_human_sequences)['Average sequence length'] // 2

generated_sequences = set()  # Use a set to store unique generated sequences
unique_sequences = []  # List to maintain the order of generation
prefixes = []

test_data = pd.read_csv("/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/seq_clusters/human_test_data.csv")

for index, row in test_data.iterrows():
    if row['prime'] == '5p':
        if relevant_seq == 'full_seq':
            prefixes.append(row['full_seq'][:int(row['Mature_end'])+1])
        else:
            prefixes.append(row['Mature'])
    else:
        if relevant_seq == 'full_seq':
            prefixes.append(row['full_seq'][:int(row['End_star'])+1])
        else:
            prefixes.append(row['Star'])

for prefix in prefixes:
    input_text = prefix
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)

    while True:
        generated = model.generate(
            input_ids,
            max_length=avg_length,
            do_sample=True,
            attention_mask=attention_mask
        )
        # Decode without skipping special tokens to find and remove them
        generated_sequence = tokenizer.decode(generated[0])
        # Check for unwanted special tokens in the middle of the sequence
        if "<SOS>" in generated_sequence[4:] or "<EOS>" in generated_sequence[:-4]:
            break
        cleaned_sequence = generated_sequence.replace(' ', '')

        # Check if the generated sequence is unique
        if cleaned_sequence not in generated_sequences:
            generated_sequences.add(cleaned_sequence)
            unique_sequences.append(cleaned_sequence)
            break

final_sequences = list(generated_sequences)

print(f"Generated {len(final_sequences)} unique sequences without the last {i} characters.")


Generated 100 unique sequences without the last 388 characters.


### Generate full seq

In [22]:
#generate without splitting

# Set logging level to error to avoid the informational logs
transformers.logging.set_verbosity_error()

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("human_ms_all_data_hairpin_mirgendb_gff") # gpt_fine_tuned_human_mirgendb_gff

# Ensure that the tokenizer's pad_token is set if it's not already
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

unique_sequences = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

avg_length = calculate_statistics(original_human_sequences)['Average sequence length'] // 2

generated_sequences = set()  # Use a set to store unique generated sequences

for i in tqdm(range(100)):
    input_text = "<SOS>"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)

    while True:
        generated = model.generate(
            input_ids,
            max_length=avg_length,
            do_sample=True,
            attention_mask=attention_mask
        )
        # Decode without skipping special tokens to find and remove them
        generated_sequence = tokenizer.decode(generated[0])

        # Check for unwanted special tokens in the middle of the sequence
        if "<SOS>" in generated_sequence[4:] or "<EOS>" in generated_sequence[:-4]:
            break

        cleaned_sequence = generated_sequence.replace(' ', '')

        # Check if the generated sequence is unique
        if cleaned_sequence not in generated_sequences:
            generated_sequences.add(cleaned_sequence)
            unique_sequences.append(cleaned_sequence)
            break

final_sequences = list(generated_sequences)

print(f"Generated {len(final_sequences)} unique sequences.")


Using pad_token, but it is not set yet.
Using eos_token, but it is not set yet.
100%|██████████| 100/100 [00:16<00:00,  5.93it/s]

Generated 100 unique sequences.


In [44]:
print(calculate_statistics(generated_sequences))

{'Number of sequences': 100, 'Average sequence length': 202, 'Min sequence length': 155, 'Max sequence length': 238}


### Generated to file

In [23]:
with open('data/generated_100_full_data_hairpin_human.txt', 'w') as f: #generated_100_hairpin_full_data_human
    for string in tqdm(generated_sequences):
        string = string.replace('<SOS>','').replace('<EOS>','')
        f.write(string + '\n')

100%|██████████| 100/100 [00:00<00:00, 699050.67it/s]


In [19]:
generated_sequences = []
with open('data/generated_100_full_data_hairpin_human.txt', 'r') as f: #generated_100_full_data_hairpin_human / generated_500_human_ms / genreated_new_10000_human_no_flanks_mirgenedb_gff_ms
    for line in f:
        # Add each line to the list after stripping newline characters
        generated_sequences.append(line.strip())